In [ ]:
# --- CoastSat cay circularity time series: annual & seasonal (FACETTED) ---
import arcpy, os, pandas as pd, numpy as np, matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ========= USER INPUTS =========
gdb = r"D:\GIS\REEF_ISLAND_RESEARCH.gdb"

# order here controls vertical facet order (top -> bottom)
feature_classes = [
    "Csat_update_podcaddi_poly",   # TOP    - Podang Podang Caddi
    "Csat_update_podlompo_poly",   # MIDDLE - Podang Podang Lompo
    "Csat_update_kodkeke_poly"     # BOTTOM - Kodingareng Keke
]
site_labels = [
    "Podang Podang Caddi",
    "Podang Podang Lompo",
    "Kodingareng Keke"
]

date_field   = "date"
circ_field   = "circ_pp"   # circularity field
out_png      = r"D:\GIS\plots\cay_circularity_timeseries_annual_seasonal_FACET.png"

point_alpha         = 0.35   # raw points transparency
annual_line_width   = 3.0
seasonal_line_width = 2.0
marker_size_raw     = 18
marker_size_season  = 40
fontsize_axes       = 14
# ===============================

arcpy.env.workspace = gdb

def fc_to_df(fc_path, site_name, date_field, value_field):
    arr = arcpy.da.FeatureClassToNumPyArray(fc_path, [date_field, value_field],
                                            null_value=np.nan)
    df = pd.DataFrame(arr)
    df[date_field]  = pd.to_datetime(df[date_field], errors="coerce",
                                     utc=True).dt.tz_localize(None)
    df[value_field] = pd.to_numeric(df[value_field], errors="coerce")
    df = df.dropna(subset=[date_field, value_field]).copy()
    df["Site"] = site_name
    return df.sort_values(by=date_field)

frames = []
for fc, label in zip(feature_classes, site_labels):
    fc_path = os.path.join(gdb, fc) if os.path.dirname(fc) == "" else fc
    if not arcpy.Exists(fc_path):
        arcpy.AddWarning(f"Not found: {fc_path}")
        continue
    frames.append(fc_to_df(fc_path, label, date_field, circ_field))

if not frames:
    raise RuntimeError("No valid feature classes found.")

df_all = (pd.concat(frames, ignore_index=True)
            .rename(columns={date_field: "Date", circ_field: "Circularity"}))

# === NEW: annual circularity stats per site ===========================

# Annual mean circularity for each site/year
ann_all = (df_all
           .groupby(["Site", df_all["Date"].dt.year.rename("Year")], as_index=False)
           ["Circularity"]
           .mean()
           .rename(columns={"Circularity": "Circ_annual"}))

print("\n=== Circularity summary from annual means ===\n")
for site in site_labels:
    sdata = ann_all[ann_all["Site"] == site]["Circ_annual"]
    if sdata.empty:
        continue

    n_years = len(sdata)
    mean_c  = sdata.mean()
    sd_c    = sdata.std(ddof=1)
    cv_c    = (sd_c / mean_c) * 100.0
    rng_c   = sdata.max() - sdata.min()

    print(site)
    print(f"  n years      = {n_years}")
    print(f"  mean circ    = {mean_c:.3f}")
    print(f"  sd circ      = {sd_c:.3f}")
    print(f"  CV           = {cv_c:.1f}%")
    print(f"  annual range = {rng_c:.3f}\n")


# ---------- different sites colour palettes ----------
#SITE_COLORS = {
#    "Podang Podang Caddi": "#2ca02c",
#    "Podang Podang Lompo": "#9467bd",
#    "Kodingareng Keke":    "#1f77b4",

# ---------- all balck sites colour palettes ----------
SITE_COLORS = {site: "#000000" for site in site_labels}

SEASON_COLORS = {"DJF": "C3", "MAM": "C1", "JJA": "C2", "SON": "C0"}
SEASON_ORDER  = ["DJF", "MAM", "JJA", "SON"]
SEASON_MIDMON = {"DJF": 1, "MAM": 4, "JJA": 7, "SON": 10}

# ---------- derive Year/Season/SeasonYear (ordered categorical) ----------
df_all["Year"]  = df_all["Date"].dt.year
df_all["Month"] = df_all["Date"].dt.month
season_str = np.select(
    [df_all["Month"].isin([12, 1, 2]),
     df_all["Month"].isin([3, 4, 5]),
     df_all["Month"].isin([6, 7, 8]),
     df_all["Month"].isin([9, 10, 11])],
    SEASON_ORDER,
    default="DJF"
)
df_all["Season"] = pd.Categorical(season_str,
                                  categories=SEASON_ORDER,
                                  ordered=True)
# put December into the next seasonal year so DJF stays together
df_all["SeasonYear"] = np.where(df_all["Month"] == 12,
                                df_all["Year"] + 1,
                                df_all["Year"])

# ---------- figure / facets ----------
n_sites = len(site_labels)
fig, axes = plt.subplots(n_sites, 1, sharex=True,
                         figsize=(10, 8), dpi=150)
if n_sites == 1:
    axes = [axes]

plt.subplots_adjust(top=0.93, bottom=0.12, hspace=0.12)

# global x-limits for all panels
xmin = df_all["Date"].min()
xmax = df_all["Date"].max()


# ============================================================
#  NUMERIC SUMMARIES: ANNUAL & SEASONAL CIRCULARITY
# ============================================================

# ---------- 1) Annual stats by site (mean of annual means) ----------
annual_means = (
    df_all
    .groupby(["Site", "Year"], as_index=False)["Circularity"]
    .mean()
    .rename(columns={"Circularity": "Circ_ann"})
)

print("\n=== Circularity summary from annual means ===\n")
for site in site_labels:
    sub = annual_means[annual_means["Site"] == site]
    if sub.empty:
        continue
    mu  = sub["Circ_ann"].mean()                        # average of annual means
    rng = sub["Circ_ann"].max() - sub["Circ_ann"].min() # max - min of annual means
    print(f"{site}: mean = {mu:.3f}, annual range = {rng:.3f}")

# ---------- 2) Seasonal stats by site and season ----------
seasonal_means = (
    df_all
    .groupby(["Site", "SeasonYear", "Season"], as_index=False)["Circularity"]
    .mean()
    .rename(columns={"Circularity": "Circ_season"})
)

print("\n=== Seasonal circularity summary from seasonal means ===\n")
for site in site_labels:
    print(f"\n{site}:")
    sub_site = seasonal_means[seasonal_means["Site"] == site]
    if sub_site.empty:
        continue
    for s in SEASON_ORDER:
        sub = sub_site[sub_site["Season"] == s]
        if sub.empty:
            continue
        mu  = sub["Circ_season"].mean()
        rng = sub["Circ_season"].max() - sub["Circ_season"].min()
        print(f"  {s}: mean = {mu:.3f}, seasonal range = {rng:.3f}")

# ---------- legend handles ----------

# (1) Season legend (top panel)
season_handles = [
    Line2D([0], [0], marker='o', color='w',
           markerfacecolor=SEASON_COLORS[s],
           markeredgecolor='black', markersize=8, label=s)
    for s in SEASON_ORDER
]

# (2) Data-type legend (middle panel)
data_type_handles = [
    Line2D([0], [0], marker='+', linestyle='None',
           color='grey', markersize=8, label='Raw data'),
    Line2D([0], [0], linestyle='-',
           color='black', linewidth=annual_line_width,
           label='Annual mean'),
    Line2D([0], [0], linestyle='--',
           color='black', linewidth=seasonal_line_width,
           label='Seasonal mean'),
]

# loop in the *desired* order using site_labels
for ax, site in zip(axes, site_labels):
    sdf = df_all[df_all["Site"] == site].copy()
    base = SITE_COLORS[site]

    # --- raw points (grey +) ---
    ax.scatter(sdf["Date"], sdf["Circularity"],
               s=marker_size_raw, color="grey", marker="+",
               alpha=point_alpha, label=None, zorder=1.5)

    # --- annual mean (solid line, site colour) ---
    ann = sdf.groupby("Year", as_index=False)["Circularity"].mean()
    ann["Date"] = pd.to_datetime(ann["Year"].astype(str) + "-01-01")
    ann = ann.sort_values("Date")
    ax.plot(ann["Date"], ann["Circularity"],
            color=base, linewidth=annual_line_width,
            linestyle="-", zorder=3)

    # --- seasonal mean (dashed line + coloured points) ---
    sea = sdf.groupby(["SeasonYear", "Season"], as_index=False)["Circularity"].mean()
    sea["Month"] = sea["Season"].map(SEASON_MIDMON).astype(int)
    sea["Date"]  = pd.to_datetime(dict(year=sea["SeasonYear"],
                                       month=sea["Month"],
                                       day=15))
    sea = sea.sort_values(["SeasonYear", "Season"])
    ax.plot(sea["Date"], sea["Circularity"],
            color=base, linewidth=seasonal_line_width,
            linestyle="--", alpha=0.9, zorder=3)

    for s in SEASON_ORDER:
        sub = sea[sea["Season"] == s]
        if len(sub):
            ax.scatter(sub["Date"], sub["Circularity"],
                       s=marker_size_season,
                       color=SEASON_COLORS[s],
                       edgecolor="black", linewidth=0.7,
                       zorder=4)

    # axes style
    ax.set_ylim(0.0, 1.0)         # same y-scale for all facets
    ax.set_xlim(xmin, xmax)
    ax.grid(True, which="major", linestyle="--", alpha=0.35)
    ax.tick_params(axis="both", which="major", labelsize=fontsize_axes)

    # site label just OUTSIDE upper-left of each facet box
    ax.text(0, 1.02, site,
            transform=ax.transAxes,
            ha="left", va="bottom",
            fontsize=14, fontweight="bold")

# y-axis label (middle panel) – applies visually to all
mid = n_sites // 2
axes[mid].set_ylabel("Circularity (0–1)", fontsize=fontsize_axes)

# x-axis label only on bottom facet
axes[-1].set_xlabel("Time", fontsize=fontsize_axes)

# ---------- legends ----------
top_ax = axes[0]
mid_ax = axes[mid]

# Season legend (stay in top facet, lower left)
season_legend = top_ax.legend(handles=season_handles, title="Season",
                              loc="lower left", frameon=True, edgecolor='k',
                              fontsize=fontsize_axes-1,
                              title_fontsize=fontsize_axes)

# Data-type legend (move to middle facet, bottom-left)
data_legend = mid_ax.legend(handles=data_type_handles,
                            loc="lower left",
                            bbox_to_anchor=(0, 0),  # <<< TUNE POSITION HERE
                            frameon=True, edgecolor='k',
                            fontsize=fontsize_axes-1)

# make sure the first legend stays on the top axes
top_ax.add_artist(season_legend)

os.makedirs(os.path.dirname(out_png), exist_ok=True)
plt.tight_layout()
plt.savefig(out_png, dpi=300)
plt.close()
print(f"Saved facetted seasonal circularity plot to: {out_png}")
